# Analyzing SASUMO outputs

### Import the Required Libraries

In [1]:
import os
import sys
import pathlib

In [2]:
def find_root(path):
    if os.path.split(path)[-1] != "airport-harper-sumo":
        return find_root(os.path.split(path)[0])
    return path


In [3]:
import pathlib

PATH = find_root(pathlib.Path().absolute())
if PATH not in sys.path:
    print(f"adding {PATH} to path")
    sys.path.append(PATH)


adding /Users/max/Development/airport-harper-sumo to path


In [4]:
import json5 as json
import sys

import numpy as np
import pandas as pd
import pint
import plotly.graph_objects as go

from SALib.analyze import sobol
from SASUMO.params import SASUMOConf, ParameterSweepConf




## Defining the Analysis Root

In [5]:
from dataclasses import dataclass


@dataclass
class FileHandler:

    root: str
    results: pd.DataFrame = None
    si_df: pd.DataFrame = None
    _p: dict = None

    @property
    def problem(
        self,
    ):
        if not self._p:
            p = os.path.join(self.root, "SALib_Problem.json")
            self._p = json.load(open(p))
        return self._p

    @property
    def results_csv(
        self,
    ):
        return os.path.join(self.root, "processed_results.csv")

    @property
    def sample_file(
        self,
    ):
        return os.path.join(self.root, "SALib_Samples.txt")

    @property
    def sasumo_conf(
        self,
    ):
        return os.path.join(self.root, "sasumo_params.yaml")

    @property
    def results_csv(
        self,
    ):
        return os.path.join(self.root, "processed_results.csv")

    def load_results(
        self,
    ):
        self.results = pd.read_csv(self.results_csv)


In [6]:
PATH = "/Users/max/tmp/InputSequenceGeneration"


In [7]:
SAs = {}
for _dir in os.scandir(PATH):
    if _dir.is_dir():
        file_handler = FileHandler(_dir.path)
        try:
            n = ParameterSweepConf.static_reload(file_handler.sasumo_conf).ParameterSweep.N
            SAs[n] = file_handler
            file_handler.load_results()
        except FileNotFoundError:
            print(
                _dir.name,
            )


## Creating a table of the Results and Parameters


### Calculating Watiting Time Ratio


In [8]:
import itertools

MAINLINE_PHASES = ["1", "2", "5", "6"]
SIDE_PHASES = ["3", "4", "7", "8"]

TL = ["63082002", "63082003", "63082004"]

MAINLINE_SUMBOIS = [
    f"{tl}_{p}_meanTimeLoss"
    for tl, p in itertools.product(TL, MAINLINE_PHASES)
    if tl != "63082002" and p not in ["1", "3", "7", "8"]
]
SIDESTREET_SUMBOIS = [
    f"{tl}_{p}_meanTimeLoss"
    for tl, p in itertools.product(TL, SIDE_PHASES)
    if tl != "63082002" and p not in ["1", "3", "7", "8"]
]


In [9]:
for n, fh in SAs.items():
    fh.results["mainline_timeloss"] = fh.results[MAINLINE_SUMBOIS].sum(axis=1)
    fh.results["sidestreet_timeloss"] = fh.results[SIDESTREET_SUMBOIS].sum(axis=1)
    fh.results["timeloss_ratio"] = fh.results["sidestreet_timeloss"] / (
        fh.results["mainline_timeloss"] + fh.results["sidestreet_timeloss"]
    )


### Creating a Side Street FC Ratio


In [10]:
# import plotly.figure_factory as ff


# fig = ff.create_distplot(
#     [
#         fh.results["Fuel_L"].values
#         for fh in SAs.values()
#     ],
#     group_labels=list(SAs.keys()),
#     bin_size=0.5,
# )
# fig.show()


## Plotting Calibration


In [11]:
for n, fh in SAs.items():
    print(
        f"{n}: Percentage that fail calibration: {(sum(fh.results.cal_score < 1) / len(fh.results)) * 100}%"
    )


128: Percentage that fail calibration: 38.28125%
32: Percentage that fail calibration: 40.625%
16: Percentage that fail calibration: 12.5%
8: Percentage that fail calibration: 12.5%
64: Percentage that fail calibration: 34.375%
1024: Percentage that fail calibration: 38.76953125%
4096: Percentage that fail calibration: 38.330078125%
256: Percentage that fail calibration: 36.71875%
2048: Percentage that fail calibration: 38.96484375%
512: Percentage that fail calibration: 38.671875%


In [12]:
SAs["2"].results

KeyError: '2'

In [13]:
considered_parameters = [
    "Fuel_L", 
    "all_delay_average",
    "all_travel_time_average",
    "timeloss_ratio"
]


In [14]:
results = {n: {param: [fh.results[param].quantile(0.05), fh.results[param].quantile(0.95)] for param in considered_parameters} for n, fh in SAs.items()}


In [15]:
results

{'128': {'Fuel_L': [429.1536126302007, 591.0069140761198],
  'all_delay_average': [26.95499018396564, 47.30578051553038],
  'all_travel_time_average': [77.51557787701847, 97.9324505771558],
  'timeloss_ratio': [0.3621218324227889, 0.5052646668592422]},
 '32': {'Fuel_L': [439.8999515097846, 581.6881052824925],
  'all_delay_average': [26.832496350115765, 48.59984225692036],
  'all_travel_time_average': [77.37267637486215, 99.2723843845324],
  'timeloss_ratio': [0.3686174536356668, 0.5008320572741789]},
 '16': {'Fuel_L': [442.1916800595052, 561.838077185761],
  'all_delay_average': [28.436029513894148, 46.067780225776815],
  'all_travel_time_average': [79.03629416387261, 97.01227057119823],
  'timeloss_ratio': [0.3701486413086346, 0.48581027140935723]},
 '8': {'Fuel_L': [424.0467960799683, 589.827090625876],
  'all_delay_average': [28.458506185874835, 44.39438343695211],
  'all_travel_time_average': [79.14043346412959, 95.44446710759125],
  'timeloss_ratio': [0.35840299409455484, 0.518601

In [25]:
json.dump(results, open(os.path.join(PATH, "quantiles.json"), "w"), indent=4, quote_keys=True, trailing_commas=False)

In [24]:
import plotly.express as px
from plotly.subplots import make_subplots


results_df = SAs['08.25.2022_17.06.15'].results


def myround(x, base=5):
    return base * round(x / base)


px_fig = px.ecdf(
    results_df,
    x="Fuel_Gal",
    markers=True,
    lines=False,
    marginal="histogram",
)


fig = make_subplots(
    rows=2, cols=2, start_cell="bottom-left", shared_yaxes=True, vertical_spacing=0.2
)

for i, (p_name, name) in enumerate(
    [
        (
            "Fuel [L]",
            "Fuel_L",
        ),
        ("Average Delay [s]", "all_delay_average"),
        ("Average Travel Time [s]", "all_travel_time_average"),
        ("Ratio of Delay", "timeloss_ratio"),
    ]
):
    px_fig = px.ecdf(
        results_df,
        x=name,
        markers=True,
        lines=False,
        marginal="histogram",
        # name="Simulation Data",
    )
    trace = px_fig.data[0]
    trace.marker.color = "black"
    trace.showlegend = i == 0
    trace.name = "SA Sample"

    fig.add_trace(trace, row=i // 2 + 1, col=i % 2 + 1)

   

    _b = 5 if results_df[name].max() > 5 else 0.1
    dtick = myround((results_df[name].max() - results_df[name].min()) / _b, _b)
    fig.update_layout(
        **{
            "xaxis"
            if i < 1
            else f"xaxis{i+1}": dict(
                title=p_name,
                nticks=10,
                showgrid=True,
                minor_showgrid=True,
                tickfont=dict(
                    size=20,
                ),
            ),
            "yaxis"
            if i < 1
            else f"yaxis{i+1}": dict(
                showgrid=True,
                minor_showgrid=True,
                ticks="outside" if i % 2 < 1 else "inside",
                title="Empirical cdf F(x)" if i % 2 < 1 else None,
                tickfont=dict(
                    size=20,
                ),
            ),
        }
    )  # range=_range, dtick=dtick,


fig.update_layout(
    template="simple_white",
    font_family="Open Sans",
    font_size=22,
    height=600,
    width=800,
    margin=dict(l=50, r=50, b=20, t=20, pad=4),
    legend=dict(yanchor="bottom", y=-0.38, xanchor="right", x=1, orientation="h"),
)


fig.update_annotations(font=dict(family="Open Sans", size=22))


fig.show()


In [51]:
import plotly.figure_factory as ff

mean = []
plus_minus = [
    [],
    []
]
labels = []


for n in range(1, 100):
    _n = n * 10
    labels.append(_n)
    _sample = results_df.sample(
        n=_n
    ).all_travel_time_average
    mean.append(_sample.mean())
    plus_minus[0].append(
        _sample.quantile(0.05)
        )
    plus_minus[1].append(
        _sample.quantile(0.95)
    )



In [52]:
fig = go.Figure()


output = "f_out"

colorscale = [
    "rgba(229, 185, 173, 1)",
    "rgba(177, 199, 179, 1)",
    "rgba(241, 234, 200, 1)",
    "rgba(0, 147, 146, 1)",
    "rgba(114, 170, 161, 1)",
    "rgba(229, 185, 173, 1)",
    "rgba(217, 137, 148, 1)",
    "rgba(208, 88, 126, 1)",
]

# row_col = [(1, 1), (1, 2), (2, 1), (2, 2)]
# for j, (p_name, name) in enumerate(zip(PRETTY_COLUMNS, SI_COLUMNS)):
fig = go.Figure()


i = 0
fig.add_trace(

    go.Scatter(
        x=labels,
        y=plus_minus[0],
        showlegend=False,
        # fill="tonexty",
        line=dict(color=colorscale[i]),
    ),
    # row=row_col[j][0],
    # col=row_col[j][1],
)
fig.add_trace(
    go.Scatter(
        x=labels,
        y=plus_minus[1],
        showlegend=False,
        line=dict(color=colorscale[i]),
        fill="tonexty",
    ),
    # row=row_col[j][0],
    # col=row_col[j][1],
)

fig.add_trace(
    go.Scatter(
        x=labels,
        y=mean,
        name="SA Sample",
        showlegend=True,
        line=dict(color=colorscale[i]),
    ),
    # row=row_col[j][0],
    # col=row_col[j][1],
    # fill="tonexty"
)

        # if row_col[j][0] == 1:

        #     fig.update_layout(
        #         **{
        #             "xaxis{}".format(j + 1): dict(
        #                 title="Number of Simulations" if row_col[j][0] == 1 else "",
        #                 dtick=10000,
        #                 tickformat="d",
        #                 tickangle=45,
        #                 # showexponent="all",
        #                 # exponentformat="e",
        #             ),
        #             "yaxis{}".format(j + 1): dict(
        #                 title=r"Sensitivity Index" if row_col[j][1] == 0 else "",
        #                 range=[0, 1],
        #             ),
        #             # "yaxis": dict(title=r"Sensitivity Index", range=[0, 1]),
        #         }
        #         # tickvals=tickvals,
        #     )

fig.update_layout(
    template="simple_white",
    # font_family="helvetica",
    font_family="Open Sans",
    title_font_family="Open Sans",
    title_font_size=24,
    font_size=24,
    height=800,
    width=800,
    barmode="stack",
    # bargap=0.15,
    legend=dict(yanchor="top", y=0.98, xanchor="right", x=1, orientation="h"),
    margin=dict(l=50, r=50, b=20, t=20, pad=4),
    # yaxis=dict(
    #     # title=r"Sensitivity Index",
    #     range=[0, 1],
    # ),
    # yaxis3=dict(
    #     title=r"Sensitivity Index",
    #     range=[0, 1],
    # ),
)

fig.update_annotations(font=dict(family="Open Sans", size=24))
fig.write_image("STs_All.png", scale=2, width=800, height=800)

fig.show()
